In [1]:
!pip install sentence_transformers pinecone-client openai python-docx

In [2]:
import os
from sentence_transformers import SentenceTransformer
from docx import Document
from pinecone import Pinecone
from openai import OpenAI

In [3]:
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
os.environ['PINECONE_API_KEY'] = "PINECONE_API_KEY"
os.environ['PINECONE_TASK_NAME'] = "PINECONE_TASK_NAME"

In [4]:
#This is for embedding. In here, one LM model from huggingface used.

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


# text ='Abc'
# model.encode(text).tolist() #exmple how to do encoding.

In [5]:
#Function to split long documents in to smaller parts
def split_text_into_chunks(plain_text, max_chars=2000):
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

In [6]:
def load_and_split_docx(file_path):
    doc = Document(file_path)
    plain_text = ""
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                try:
                    plain_text += cell.text + "\n"
                except UnicodeEncodeError:
                    print("Cannot display this character.")

    chunks = split_text_into_chunks(plain_text)
    return chunks

In [7]:
#Todo: Initialization of vector database module
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'] )
index = pc.Index(os.environ['PINECONE_TASK_NAME']) #Todo: Fill out with index name.

In [8]:
insertion = []

def addData(corpusData):
    id = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk = corpusData[i]
        chunkInfo = {
          "id": str(id + i),
          "values": model.encode(chunk).tolist(),
          "metadata": {'context': chunk}
        }
        insertion.append(chunkInfo)

    index.upsert(vectors=insertion)
    print(insertion)

In [9]:
chunks = load_and_split_docx("DataLaw.docx")
addData(chunks)

[{'id': '580', 'values': [-0.029314115643501282, 0.0663209855556488, -0.05352400988340378, 0.021750271320343018, 0.06200535595417023, -0.029395578429102898, 0.0026770748663693666, 0.12195467203855515, -0.02931365929543972, -0.016212934628129005, 0.03781656548380852, 0.003695105202496052, -0.029208848252892494, 0.027271300554275513, -0.06400725990533829, -0.02130632847547531, -0.017326191067695618, 0.10697995871305466, -0.05995751544833183, 0.06344587355852127, 0.100774846971035, 0.051006071269512177, 0.03939034417271614, -0.038785818964242935, 0.0024790323805063963, -0.07519909739494324, -0.029255295172333717, -0.0033278074115514755, 0.006175843067467213, -0.0315210185945034, -0.005912788677960634, 0.09169455617666245, 0.026379941031336784, 0.010197685100138187, 0.06886374950408936, -0.051172319799661636, 0.03929213434457779, -0.07280312478542328, -0.0465703010559082, -0.024940865114331245, 0.07983662933111191, -0.03737629950046539, -0.10038937628269196, 0.025152938440442085, -0.064996

In [27]:
def create_prompt(matching_contexts, query):
    context_str = '\n'.join([f"Context {i+1} ({score}): {context}" for i, (context, score) in enumerate(matching_contexts)])
    prompt = f"Contexts:\n{context_str}\nUser Question: {query}\nAnswer:"
    return prompt


In [28]:
def generate_answer(prompt):
    # Todo: Pass the generated prompt and pass it to gpt-3 to get answers.
    response = OpenAI(api_key=os.environ['OPENAI_API_KEY']).completions.create(
            model="text-embedding-ada-002",
            prompt=prompt
        )
    return response.choices[0].text.strip()

In [29]:
def user_query(query):
    # Find matching contexts with similarity scores
    matching_contexts = find_match(query, 10)

    # Extract only the contexts from the tuples
    matching_contexts = [context for context, _ in matching_contexts]

    # Generate prompt based on matching contexts and user query
    prompt = create_prompt(matching_contexts, query)

    # Get answer from GPT-3
    answer = generate_answer(prompt)

    return answer

In [30]:
def find_match(query, k):
    query_em = model.encode(query).tolist()
    result = index.query(vector=query_em, top_k=k, include_metadata=True)

    return [(m.metadata.context, m.score) for m in result['matches'] if m.metadata is not None and hasattr(m.metadata, 'context')]


In [34]:
user_query("How can I do this?")

'1:2:1 has 1:2:1Click9Cr'